In [89]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re

In [90]:
# Get the actors names

df  = pd.read_csv('../../dataset/nou_hearings_full.csv')
actors = df["actor"].unique()
base_url = "https://w2.brreg.no/enhet/sok/"


In [91]:
url_list = {}

for actor in actors:
    print(actor)
    actor_processed = actor.replace(" ", "%20")
    url = f"https://w2.brreg.no/enhet/sok/treffliste.jsp?navn={actor_processed}&orgform=0&fylke=0&kommune=0"
    response = requests.get(url)
    org_link = None

    # Parse the response
    soup = bs(response.text, 'html.parser')
    try:
        # Find the link to the actor
        pattern = re.compile(f".*{actor.upper()}.*")
        link = soup.find("a", text=pattern).get("href")
        org_link = base_url + link
    except:
        # Remove the last two characters of the actor name
        actor_new = actor[:-2]
        try:
            # Find the link to the actor
            pattern = re.compile(f".*{actor_new.upper()}.*")
            link = soup.find("a", text=pattern).get("href")
            org_link = base_url + link
        except:
            print(f"Could not find {actor} in brønnøysundregistrene")
            
    
    url_list[actor] = org_link
url_list

Abelia


C:\Users\henriklarsen\AppData\Local\Temp\ipykernel_20284\71157717.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  link = soup.find("a", text=pattern).get("href")


Advokatforeningen


C:\Users\henriklarsen\AppData\Local\Temp\ipykernel_20284\71157717.py:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  link = soup.find("a", text=pattern).get("href")


Agder fylkeskommune
Akademikerne
Alliansen ny landbrukspolitikk
Animalia AS
Ann Norderhaug
Arbeidsgiverforeningen Spekter
Arkitektorganisasjonene felles
Could not find Arkitektorganisasjonene felles in brønnøysundregistrene
Arne Aasen
Asker kommune
Askøy kommune
Bergen INP
Bergen kommune
Besteforeldrenes klimaaksjon
Besteforeldrenes klimaaksjon, samferdselsgruppa
Could not find Besteforeldrenes klimaaksjon, samferdselsgruppa in brønnøysundregistrene
Biogass Norge
Bioteknologirådet
BirdLife Norge
Bodø kommune
Boligprodusentenes Forening
Buskerud fylkeskommune
Bærum kommune
Dag Hendrik Flølo
Could not find Dag Hendrik Flølo in brønnøysundregistrene
Den norske kyrkja ved Biskopen i Møre Ingeborg Midttømme
Could not find Den norske kyrkja ved Biskopen i Møre Ingeborg Midttømme in brønnøysundregistrene
Direktoratet for samfunnssikkerhet og beredskap
DNMF
Drammen kommune
Eidsvoll INP
Elkem ASA
Energigass Norge
Equinor ASA
Etne Industri- og næringspartiet
Could not find Etne Industri- og næri

{'Abelia': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=983489060',
 'Advokatforeningen': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=936575668',
 'Agder fylkeskommune': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=921707134',
 'Akademikerne': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=979371349',
 'Alliansen ny landbrukspolitikk': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=919764244',
 'Animalia AS': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=919117060',
 'Ann Norderhaug': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=996348008',
 'Arbeidsgiverforeningen Spekter': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=871281602',
 'Arkitektorganisasjonene felles': None,
 'Arne Aasen': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=980002748',
 'Asker kommune': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=920125298',
 'Askøy kommune': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=964338442',
 'Bergen INP': 'https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr=925520

In [92]:
# Save the urls to a csv
df = pd.DataFrame(list(url_list.items()), columns=["actor","url"])
print(df)
df.to_csv("actor_urls.csv")

                                         actor  \
0                                       Abelia   
1                            Advokatforeningen   
2                          Agder fylkeskommune   
3                                 Akademikerne   
4               Alliansen ny landbrukspolitikk   
..                                         ...   
207                      WWF Verdens naturfond   
208                                       Yara   
209  Yrkesorganisasjonenes Sentralforbund (YS)   
210                                     Z-Clix   
211                                       ZERO   

                                                   url  
0    https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr...  
1    https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr...  
2    https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr...  
3    https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr...  
4    https://w2.brreg.no/enhet/sok/detalj.jsp?orgnr...  
..                                                 ...  


In [96]:
from operator import contains


df = pd.read_csv("actor_urls.csv")


for i, row in df.iterrows():
    actor = row["actor"]
    url = row["url"]
    
    try:
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')
        org_form = soup.find_all("div", class_="row")[4].find("div", class_="col-sm-8").find("p").get_text()

        org_forms = org_form.split("/")
        print(org_forms)
        df.at[i, "org_form"] = org_forms[0]
        
    except:
        print(f"Could not find org number for {actor}")
        df.at[i, "org_form"] = None

['Forening', 'lag', 'innretning']
['Forening', 'lag', 'innretning']
['Fylkeskommune']
['Forening', 'lag', 'innretning']
['Forening', 'lag', 'innretning']
['Aksjeselskap']
['Enkeltpersonforetak']
['Forening', 'lag', 'innretning']
Could not find org number for Arkitektorganisasjonene felles
['Enkeltpersonforetak']
['Kommune']
['Kommune']
['Forening', 'lag', 'innretning']
['Kommune']
['Forening', 'lag', 'innretning']
Could not find org number for Besteforeldrenes klimaaksjon, samferdselsgruppa
['Forening', 'lag', 'innretning']
['Organisasjonsledd']
['Forening', 'lag', 'innretning']
['Kommune']
['Forening', 'lag', 'innretning']
['Fylkeskommune']
['Kommune']
Could not find org number for Dag Hendrik Flølo
Could not find org number for Den norske kyrkja ved Biskopen i Møre Ingeborg Midttømme
['Organisasjonsledd']
['Forening', 'lag', 'innretning']
['Kommune']
['Forening', 'lag', 'innretning']
['Allmennaksjeselskap']
['Forening', 'lag', 'innretning']
['Allmennaksjeselskap']
Could not find org 

In [97]:
df.to_csv("actor_urls.csv")